# FOMC Members by month
This notebook scrapes information from FRASER and the Federal Reserve Website for the membership and voting-status of the FOMC for each month of the year.

FOMC Members with Voting Rights: 
- FOMC Chair (Chair of the Federal Reserve)
- FOMC Vice Chair (Always the president of the New York Federal Bank)
- Vice Chair of the Federal Reserve
- Presidents of Voting Banks

<br>FOMC Members with No Voting Rights:
- Presidents of non-voting Federal Banks

# Imports

In [66]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import re
import requests

from datetime import datetime
from bs4 import BeautifulSoup
from dateutil.parser import parse

## FOMC Structure Calculator (2006 onwards)
FOMC structure prior to 2006 has a couple of exceptions and you will need to refer to the exact membership on the Federal Reserve's website.

In [67]:
def get_FOMC_structure_by_year(year):
    assert isinstance(year, int), 'year argument must be an integer.'
    assert year >= 2006, 'year argument must be 2006 or larger'
    delta = year - 1997
    NY = 'New York'

    def blk134(delta):
        if delta % 3 == 0:
            blk1_e = 'Richmond'
            blk3_e = 'Atlanta'
            blk4_e = 'San Francisco'
            blk1_a = 'Boston'
            blk3_a = 'St. Louis'
            blk4_a = 'Kansas City'

        elif (delta - 1) % 3 ==0:
            blk1_e = 'Boston'
            blk3_e = 'St. Louis'
            blk4_e = 'Kansas City'
            blk1_a = 'Philadelphia'
            blk3_a = 'Dallas'
            blk4_a = 'Minneapolis'

        elif (delta - 2) % 3 ==0:
            blk1_e = 'Philadelphia'
            blk3_e = 'Dallas'
            blk4_e = 'Minneapolis'
            blk1_a = 'Richmond'
            blk3_a = 'Atlanta'
            blk4_a = 'San Francisco'
        
        return blk1_e, blk3_e, blk4_e, blk1_a, blk3_a, blk4_a
    
    def blk2(delta):
        if delta % 2 == 0:
            blk2_e = 'Chicago'
            blk2_a = 'Cleveland'

        elif (delta -1) % 2 ==0:
            blk2_e = 'Cleveland'
            blk2_a = 'Chicago'

        return blk2_e, blk2_a
    
    blk1_e, blk3_e, blk4_e, blk1_a, blk3_a, blk4_a = blk134(delta)
    blk2_e, blk2_a = blk2(delta)

    return((NY, blk1_e, blk2_e, blk3_e, blk4_e),(NY, blk1_a, blk2_a, blk3_a, blk4_a))

In [68]:
get_FOMC_structure_by_year(2023)

(('New York', 'Philadelphia', 'Chicago', 'Dallas', 'Minneapolis'),
 ('New York', 'Richmond', 'Cleveland', 'Atlanta', 'San Francisco'))

## FOMC Membership up to Current Year

In [69]:
from_2006_df = pd.DataFrame()
for year in range(2006, datetime.now().year+1):
    elected_members, alternate_members = get_FOMC_structure_by_year(year)
    elec_df = pd.DataFrame({'year': [year],
                           'membership': ['elected'],
                           'ny': [elected_members[0]],
                           'blk1': [elected_members[1]],
                           'blk2': [elected_members[2]],
                           'blk3': [elected_members[3]],
                           'blk4': [elected_members[4]],
                           'combined': [list(elected_members)]
                           })
    alt_df = pd.DataFrame({'year': [year],
                           'membership': ['alternate'],
                           'ny': [alternate_members[0]],
                           'blk1': [alternate_members[1]],
                           'blk2': [alternate_members[2]],
                           'blk3': [alternate_members[3]],
                           'blk4': [alternate_members[4]],
                           'combined': [list(alternate_members)]
                           })
    
    from_2006_df = pd.concat([from_2006_df, elec_df, alt_df], axis=0)
    fomc_df = from_2006_df.reset_index(drop=True)

## Federal Bank Personnel Information
* [FRASER](https://fraser.stlouisfed.org/timeline) (Bank President Timeline)

In [70]:
def get_dates_from_str(s : str):
    s = re.sub('[^A-Za-z0-9]+', ' ', s).strip(). split(' to ')

    try:
        s = s[-2:]
    except:
        pass

    date_list = []
    for i in s:
        date_list.append(parse(str(i), fuzzy=True).strftime('%d-%m-%Y'))
    
    return date_list

In [71]:
def get_fed_bank_president_info(url: str):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    info_table = soup.find_all('div', {'class': 'row event-row'})

    pres_dict= dict()
    for i in info_table:
        # skip interim presidents
        if (i.find('h2').get_text().lower().find('interim') != -1) or (i.find('p').get_text().lower().find('interim') != -1):
            continue

        name = i.find('h2').get_text().split('|')[1].strip()

        i.p.b.clear()
        date_text = i.p.get_text().lower()

        dates = get_dates_from_str(date_text)

        if pres_dict.get(name):
            pres_dict[name].append(dates)

        else:
            pres_dict[name] = [dates]

    return pres_dict

In [72]:
fraser_fed_pres_urls = {'Atlanta': 'https://fraser.stlouisfed.org/timeline/presidents-frb-atlanta',
                        'Boston': 'https://fraser.stlouisfed.org/timeline/presidents-frb-boston',
                        'Chicago': 'https://fraser.stlouisfed.org/timeline/presidents-frb-chicago',
                        'Cleveland': 'https://fraser.stlouisfed.org/timeline/presidents-frb-cleveland',
                        'Dallas': 'https://fraser.stlouisfed.org/timeline/presidents-frb-dallas',
                        'Kansas City': 'https://fraser.stlouisfed.org/timeline/presidents-frb-kansascity',
                        'Minneapolis': 'https://fraser.stlouisfed.org/timeline/presidents-frb-minneapolis',
                        'New York': 'https://fraser.stlouisfed.org/timeline/presidents-frb-newyork',
                        'Philadelphia': 'https://fraser.stlouisfed.org/timeline/presidents-frb-philadelphia',
                        'Richmond': 'https://fraser.stlouisfed.org/timeline/presidents-frb-richmond',
                        'San Francisco': 'https://fraser.stlouisfed.org/timeline/presidents-frb-sanfrancisco',
                        'St. Louis': 'https://fraser.stlouisfed.org/timeline/presidents-frb-stlouis',
                        }

In [73]:
fed_pres_dict = dict()
for k,v in fraser_fed_pres_urls.items():
    print(k, end=" ")
    fed_pres_dict[k] = get_fed_bank_president_info(v)
    print('Done!')

Atlanta Done!
Boston Done!
Chicago Done!
Cleveland Done!
Dallas Done!
Kansas City Done!
Minneapolis Done!
New York Done!
Philadelphia Done!
Richmond Done!
San Francisco Done!
St. Louis Done!


### Converting to dataframe

In [74]:
fed_pres_df = pd.DataFrame(columns = ['Name', 'Position', 'Organization', 'Start Date', 'End Date'])
for bank, people in fed_pres_dict.items():
    for name, dates_list in people.items():
        for dates in dates_list:
            if len(dates) >1:
                end_date = dates[1]
            else:
                end_date = None
            _df = pd.DataFrame({'Name': [name],
                                'Position': ['President'],
                                'Organization': [bank],
                                'Start Date': [pd.to_datetime(dates[0], format='%d-%m-%Y')],
                                'End Date': [pd.to_datetime(end_date, format='%d-%m-%Y')],
                                })
            fed_pres_df = pd.concat([fed_pres_df, _df], axis=0)

fed_pres_df = fed_pres_df.reset_index(drop=True)     

C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\3388351948.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fed_pres_df = pd.concat([fed_pres_df, _df], axis=0)


In [75]:
fed_pres_df.head()

,Name,Position,Organization,Start Date,End Date
0,Joseph A. McCord,President,Atlanta,1914-11-16,1919-03-01
1,M. B. Wellborn,President,Atlanta,1919-03-01,1927-12-31
2,Eugene R. Black,President,Atlanta,1928-01-13,1933-05-18
3,Eugene R. Black,President,Atlanta,1934-08-16,1934-12-19
4,Oscar Newton,President,Atlanta,1935-01-16,1939-02-13


## Federal Reserve Board Website

* Chairs - table 7
* Vice Chairs - table 9
* Vice Chair for Supervision - table 11
* Governors - table 13

In [76]:
frb_url = 'https://www.federalreserve.gov/aboutthefed/bios/board/boardmembership.htm'
frb_web = requests.get(frb_url).text
soup = BeautifulSoup(frb_web, 'html.parser')

### Chair

In [77]:
table = 7
c_df = pd.DataFrame()
for person in soup.find_all('tbody')[table].find_all('tr'):
    try:
        person.sup.decompose()
    except:
        pass

    name = person.find_all('td')[1].text.strip()
    date_list = person.find_all('td')[0].text.strip().split('-')
    start_date = parse(str(date_list[0]), fuzzy=True)
    try:
        end_date = parse(str(date_list[1]), fuzzy=True)
    except:
        end_date = None
    temp_df = pd.DataFrame({'Name': [name],
                            'Position': 'Chair',
                            'Organization': 'Board of Governors',
                            'Start Date': pd.to_datetime(start_date),
                            'End Date': pd.to_datetime(end_date),
                            })

    c_df = pd.concat([c_df, temp_df], axis=0)
    c_df = c_df.reset_index(drop=True)

C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\3215023961.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  c_df = pd.concat([c_df, temp_df], axis=0)


### Vice Chair

In [78]:
table = 9
vc_df = pd.DataFrame()
for person in soup.find_all('tbody')[table].find_all('tr'):
    try:
        person.sup.decompose()
    except:
        pass

    name = person.find_all('td')[1].text.strip()
    date_list = person.find_all('td')[0].text.strip().split('-')
    start_date = parse(str(date_list[0]), fuzzy=True)
    try:
        end_date = parse(str(date_list[1]), fuzzy=True)
    except:
        end_date = None
    temp_df = pd.DataFrame({'Name': [name],
                            'Position': 'Vice Chair',
                            'Organization': 'Board of Governors',
                            'Start Date': pd.to_datetime(start_date),
                            'End Date': pd.to_datetime(end_date),
                            })

    vc_df = pd.concat([vc_df, temp_df], axis=0)
    vc_df = vc_df.reset_index(drop=True)

C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\1120463842.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vc_df = pd.concat([vc_df, temp_df], axis=0)


### Vice Chair (Supervision)

In [79]:
table = 11
vcs_df = pd.DataFrame()
for person in soup.find_all('tbody')[table].find_all('tr'):
    try:
        person.sup.decompose()
    except:
        pass

    name = person.find_all('td')[1].text.strip()
    date_list = person.find_all('td')[0].text.strip().split('-')
    start_date = parse(str(date_list[0]), fuzzy=True)
    try:
        end_date = parse(str(date_list[1]), fuzzy=True)
    except:
        end_date = None
    temp_df = pd.DataFrame({'Name': [name],
                            'Position': 'Vice Chair',
                            'Organization': 'Board of Governors',
                            'Start Date': pd.to_datetime(start_date),
                            'End Date': pd.to_datetime(end_date),
                            })

    vcs_df = pd.concat([vcs_df, temp_df], axis=0)
    vcs_df = vcs_df.reset_index(drop=True)

C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\1593844721.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vcs_df = pd.concat([vcs_df, temp_df], axis=0)


### Governors (Board Members)

In [80]:
table = 13
board_df_raw = pd.DataFrame()
for person in soup.find_all('tbody')[table].find_all('tr'):
    try:
        person.sup.decompose()
    except:
        pass
    
    start_date = person.find_all('td')[1].text.strip()
    # name = person.find_all('td')[0].text.strip().split('\n')[0]
    name = person.find_all('td')[0].text.strip().split('\r')[0]
    end_date_raw = ' ' + person.find_all('td')[2].text.strip().lower()
    end_date_raw = re.sub(r'(\s[a-zA-Z]{3,4}\.)', lambda matchobj: matchobj.group()[:-1], end_date_raw)
    end_date_raw = end_date_raw.replace(';', '.').replace(u'\xa0', u' ').replace('term began', 'appointed')

    temp_df = pd.DataFrame({'Name': [name],
                            'Position': 'Governor',
                            'Organization': 'Board of Governors',
                            'Start Date': pd.to_datetime(start_date),
                            'Other Information': end_date_raw,
                            })

    board_df_raw = pd.concat([board_df_raw, temp_df], axis=0)
    board_df_raw = board_df_raw.reset_index(drop=True)


In [81]:
board_df_raw

,Name,Position,Organization,Start Date,Other Information
0,W.G. McAdoo,Governor,Board of Governors,1913-12-23,"dec 15, 1918"
1,John Skelton Williams,Governor,Board of Governors,1914-02-02,"mar 2, 1921"
2,Charles S. Hamlin,Governor,Board of Governors,1914-08-10,reappointed in 1916 and 1926. served until fe...
3,Paul M. Warburg,Governor,Board of Governors,1914-08-10,"term expired aug 9, 1918."
4,Frederic A. Delano,Governor,Board of Governors,1914-08-10,"resigned july 21, 1918."
...,...,...,...,...,...
108,Christopher J. Waller,Governor,Board of Governors,2020-12-18,
109,Lisa D. Cook,Governor,Board of Governors,2022-05-23,"reappointed sept 13, 2023."
110,Philip N. Jefferson,Governor,Board of Governors,2022-05-23,
111,Michael S. Barr,Governor,Board of Governors,2022-07-19,


In [82]:
def process_board_df_raw(df):
    temp_df = pd.DataFrame()

    for i in df.iterrows():
        name = i[1]['Name']

        ##### Handling empty info string #####
        if len(i[1]['Other Information']) == 1:
            start_date = i[1]['Start Date']
            end_date = pd.to_datetime(None)
            _df = pd.DataFrame({'Name': [name],
                                'Position': 'Governor',
                                'Organization': 'Board of Governors',
                                'Start Date': pd.to_datetime(start_date),
                                'End Date': end_date,
                                })
            
            temp_df = pd.concat([temp_df, _df], axis=0)

        ##### Handling non-empty info string #####
        l = i[1]['Other Information'].split('.')

        if l[-1] == '' or l[-1] == ' ':
            l = l[:-1]
        
        # Handling of single specified date
        if len(l) == 1:
            if l[0].find('appointed') == -1:
                end_date = parse(l[0], fuzzy=True)
            else:
                end_date = pd.to_datetime(None)

            start_date = i[1]['Start Date']
            _df = pd.DataFrame({'Name': [name],
                                'Position': 'Governor',
                                'Organization': 'Board of Governors',
                                'Start Date': pd.to_datetime(start_date),
                                'End Date': end_date,
                                })
            temp_df = pd.concat([temp_df, _df], axis=0)

        # Handling more than 1 specified date
        elif len(l) > 1:
            end_flag = False
            start_date = i[1]['Start Date']

            for j in l:
                if (j.find('appointed') != -1):
                    if end_flag is False:
                        continue
                    if end_flag is True:
                        start_date = parse(j, fuzzy=True)
                        end_flag = False
                        continue
                else:
                    end_date = parse(j, fuzzy=True)
                    _df = pd.DataFrame({'Name': [name],
                                'Position': 'Governor',
                                'Organization': 'Board of Governors',
                                'Start Date': pd.to_datetime(start_date),
                                'End Date': end_date,
                                })
                    
                    end_flag = True

                temp_df = pd.concat([temp_df, _df], axis=0)

    return temp_df.reset_index(drop=True)

In [83]:
board_df = process_board_df_raw(board_df_raw)

C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\518604010.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, _df], axis=0)
C:\Users\sjdud\AppData\Local\Temp\ipykernel_5664\518604010.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, _df], axis=0)


### Combining Fed Bank Presidents, FRB Chair, Vice Chair, Governors into a single Dataframe

In [84]:
frb_all_df = pd.concat([c_df, vc_df, vcs_df, board_df], axis=0).sort_values(['Name', 'Start Date'], ascending=True).reset_index(drop=True)
combined_df = pd.concat([frb_all_df, fed_pres_df], axis=0).reset_index(drop=True)
combined_df

,Name,Position,Organization,Start Date,End Date
0,"A.L. Mills, Jr.",Governor,Board of Governors,1952-02-18,1965-02-28
1,Adolph C. Miller,Governor,Board of Governors,1914-08-10,1936-02-03
2,Adriana D. Kugler,Governor,Board of Governors,2023-09-13,NaT
3,Alan Greenspan,Chair,Board of Governors,1987-08-11,2006-01-31
4,Alan Greenspan,Governor,Board of Governors,1987-08-11,2006-01-31
...,...,...,...,...,...
283,Theodore H. Roberts,President,St. Louis,1983-02-01,1984-12-31
284,Thomas C. Melzer,President,St. Louis,1985-06-01,1998-01-30
285,William Poole,President,St. Louis,1998-03-23,2008-03-31
286,James Bullard,President,St. Louis,2008-04-01,2023-08-14


# Processing and sorting member information by Month

In [85]:
def get_FOMC_member_info(df, month, fomc_df=fomc_df):
    month = pd.to_datetime(month)
    select_df = df.copy(deep=True)

    # Filling Nat End Dates with today's date for filtering purposes
    select_df['End Date'] = pd.to_datetime(select_df['End Date'].fillna(pd.Timestamp('now').date()))

    # Filtering FOMC structure for selected month
    fomc_select_df = fomc_df.loc[fomc_df['year'] == month.year].reset_index(drop=True)

    # Filtering active Fed personnel in selected month
    select_df = select_df[(select_df['Start Date'] <= (month + MonthEnd())) & (select_df['End Date'] >= month)].reset_index(drop=False)

    # Dropping governor entries of personnel who were Chair and Vice Chair
    duplicate_names = select_df[(select_df['Position'] == 'Chair') | (select_df['Position'] == 'Vice Chair')]['Name']

    for name in duplicate_names:
        duplicates_mask = (select_df['Name'] == name)
        duplicates_df = select_df[duplicates_mask]
        index_to_drop = duplicates_df[duplicates_df['Position'] == 'Governor'].index

        try:
            select_df = select_df.drop(index_to_drop[0])
        except:
            pass

    # Populating FOMC position based on Fed position
    condlist = [select_df['Position'] == 'Chair',
                select_df['Organization'] == 'Board of Governors',
                (select_df['Position'] == 'President') & (select_df['Organization'] == 'New York'),
                select_df['Organization'].isin(fomc_select_df.loc[0, 'combined'][1:]),
                ]
    choicelist = ['Chair', 'Voter', 'Vice Chair', 'Voter']

    select_df['FOMC_position'] = ''
    select_df['FOMC_position'] = np.select(condlist=condlist, choicelist=choicelist, default='Non-voter')

    # Forming Output dataframe
    output_df = pd.concat([select_df[select_df['Position'] == 'Chair'],
                           select_df[select_df['Position'] == 'Vice Chair'],
                           select_df[select_df['Position'] == 'Governor'].sort_values('Name'),
                           select_df[select_df['FOMC_position'] == 'Vice Chair'],
                           select_df[(select_df['FOMC_position'] == 'Voter') & (select_df['Organization'] != 'Board of Governors')].sort_values(['Organization', 'Name']),     
                           select_df[select_df['FOMC_position'] == 'Non-voter'].sort_values(['Organization', 'Name'])
                           ]).set_index('index')
    
    # Restoring original NaT 'End Date' to output dataframe
    temp_df = df[['Name', 'End Date']]
    for i in output_df.index:
        output_df.at[i, 'End Date'] = temp_df.at[i, 'End Date']

    output_df = output_df.rename(columns={'Position': 'FRB Position', 'FOMC_position': 'FOMC Position'})

    return output_df.reset_index(drop=True)

## Getting Fed & FOMC member information by month

In [86]:
date_range = pd.date_range(start='Jan-2006', end='now', freq='MS')

FOMC_member_dict = {}
for date in date_range:
    FOMC_member_dict[date.strftime('%b-%Y')] = get_FOMC_member_info(combined_df, date)

In [87]:
FOMC_member_dict['Jan-2006']

,Name,FRB Position,Organization,Start Date,End Date,FOMC Position
0,Alan Greenspan,Chair,Board of Governors,1987-08-11,2006-01-31,Chair
1,"Roger W. Ferguson, Jr.",Vice Chair,Board of Governors,1999-10-05,2006-04-28,Voter
2,Donald L. Kohn,Governor,Board of Governors,2002-08-05,2010-09-01,Voter
3,Mark W. Olson,Governor,Board of Governors,2001-12-07,2006-06-30,Voter
4,Susan S. Bies,Governor,Board of Governors,2001-12-07,2007-03-30,Voter
5,Timothy F. Geithner,President,New York,2003-11-17,2009-01-27,Vice Chair
6,Jack Guynn,President,Atlanta,1996-01-01,2006-10-01,Voter
7,Sandra Pianalto,President,Cleveland,2003-02-01,2014-05-31,Voter
8,Jeffrey M. Lacker,President,Richmond,2004-08-01,2017-04-04,Voter
9,Janet L. Yellen,President,San Francisco,2004-06-14,2010-10-04,Voter


In [90]:
# FOMC membership (latest Month)
current_month = datetime.now().strftime('%b-%Y')
print(f'FRB/FOMC membership for {current_month}.')
FOMC_member_dict[current_month]

FRB/FOMC membership for Jun-2024.


,Name,FRB Position,Organization,Start Date,End Date,FOMC Position
0,Jerome H. Powell,Chair,Board of Governors,2018-02-05,NaT,Chair
1,Michael S. Barr,Vice Chair,Board of Governors,2022-07-19,NaT,Voter
2,Philip N. Jefferson,Vice Chair,Board of Governors,2023-09-13,NaT,Voter
3,Adriana D. Kugler,Governor,Board of Governors,2023-09-13,NaT,Voter
4,Christopher J. Waller,Governor,Board of Governors,2020-12-18,NaT,Voter
5,Lisa D. Cook,Governor,Board of Governors,2022-05-23,NaT,Voter
6,Michelle W. Bowman,Governor,Board of Governors,2018-11-26,NaT,Voter
7,John C. Williams,President,New York,2018-06-18,NaT,Vice Chair
8,Raphael Bostic,President,Atlanta,2017-06-05,NaT,Voter
9,Loretta J. Mester,President,Cleveland,2014-06-01,NaT,Voter
